In [1]:
!pip install kagglehub

In [2]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 34.0 MB/s eta 0:00:00


In [3]:
!pip install opencv-python-headless

In [4]:
import kagglehub
import pandas as pd
import numpy as np
import os
from ultralytics import YOLO
import cv2
import shutil
from google.colab import drive
from google.colab import files
import albumentations as A

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


# Data Prep from kaggle and Drive

In [5]:
print("Downloading dataset:")
path = kagglehub.dataset_download("ismailnasri20/driver-drowsiness-dataset-ddd")

Using Colab cache for faster access to the 'driver-drowsiness-dataset-ddd' dataset.


In [6]:
print("Dataset downloaded to:", path)

Dataset downloaded to: /kaggle/input/driver-drowsiness-dataset-ddd


In [7]:
for root, dir, files in os.walk(path):
  print(f"Folders : {root}")

Folders : /kaggle/input/driver-drowsiness-dataset-ddd
Folders : /kaggle/input/driver-drowsiness-dataset-ddd/Driver Drowsiness Dataset (DDD)
Folders : /kaggle/input/driver-drowsiness-dataset-ddd/Driver Drowsiness Dataset (DDD)/Non Drowsy
Folders : /kaggle/input/driver-drowsiness-dataset-ddd/Driver Drowsiness Dataset (DDD)/Drowsy


In [8]:
drive.mount('/content/drive')

if os.path.exists("new_data"):
    shutil.rmtree("new_data")

os.makedirs("new_data/train/Drowsy", exist_ok=True)
os.makedirs("new_data/train/Non_Drowsy", exist_ok=True)
os.makedirs("new_data/val/Drowsy", exist_ok=True)
os.makedirs("new_data/val/Non_Drowsy", exist_ok=True)

Mounted at /content/drive


In [9]:
#kaggle data
for path, _, files in os.walk("/kaggle/input/driver-drowsiness-dataset-ddd/Driver Drowsiness Dataset (DDD)"):
    for file in files:
        src = os.path.join(path, file)
        if "Non Drowsy" in path:
            label = "Non_Drowsy"
        else:
            label = "Drowsy"

        if file[0].upper() in ('A', 'B', 'C', 'D', 'E'):
            dst = f"new_data/val/{label}"
        else:
            dst = f"new_data/train/{label}"
        shutil.copy(src, dst)

In [10]:
#Drive data
base_path = '/content/drive/MyDrive/DDD/driver monitoring'
drowsy_path = os.path.join(base_path, 'Drowsy')
non_drowsy_path = os.path.join(base_path, 'Non Drowsy')

In [11]:
print("Drowsy samples:", len(os.listdir(drowsy_path)))
print("Non Drowsy samples:", len(os.listdir(non_drowsy_path)))

Drowsy samples: 787
Non Drowsy samples: 697


In [12]:
for file in os.listdir(drowsy_path):
    src = os.path.join(drowsy_path, file)
    dst = os.path.join("new_data/train/Drowsy", file)
    if not os.path.exists(dst):
        shutil.copy(src, dst)

In [13]:
for file in os.listdir(non_drowsy_path):
    src = os.path.join(non_drowsy_path, file)
    dst = os.path.join("new_data/train/Non_Drowsy", file)
    if not os.path.exists(dst):
        shutil.copy(src, dst)

In [14]:
for folder in [
    "new_data/train/Drowsy",
    "new_data/train/Non_Drowsy",
    "new_data/val/Drowsy",
    "new_data/val/Non_Drowsy"
]:
    print(folder, ":", len(os.listdir(folder)))

new_data/train/Drowsy : 19933
new_data/train/Non_Drowsy : 16076
new_data/val/Drowsy : 3202
new_data/val/Non_Drowsy : 4066


# Augmentation

In [16]:
transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.7),
    A.Rotate(limit=10, p=0.5),
    A.GaussianBlur(p=0.6),
    A.RandomGamma(p=0.3),
    A.Resize(224, 224),
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0, rotate_limit=0, p=0.5),
])

/usr/local/lib/python3.12/dist-packages/albumentations/core/validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)


In [17]:
train_folders = [
    "new_data/train/Drowsy",
    "new_data/train/Non_Drowsy"
]

In [18]:
for folder in train_folders:
    print(f"Augmenting images in: {folder}")
    files = os.listdir(folder)
    for img_name in files:
        img_path = os.path.join(folder, img_name)
        img = cv2.imread(img_path)
        if img is None:
            continue
        augmented = transform(image=img)["image"]

        # save in the SAME folder with new filename
        base, ext = os.path.splitext(img_name)
        save_path = os.path.join(folder, f"aug_{base}{ext}")
        cv2.imwrite(save_path, augmented)

Augmenting images in: new_data/train/Drowsy
Augmenting images in: new_data/train/Non_Drowsy


In [19]:
for folder in ["new_data/train/Drowsy", "new_data/train/Non_Drowsy"]:
    print(folder, len(os.listdir(folder)))

new_data/train/Drowsy 39866
new_data/train/Non_Drowsy 32152


# Yolo Model

In [20]:
model2 = YOLO("yolo11s-cls.pt")  # nano model

In [21]:
results = model2.train(
    data= "new_data",epochs=4, batch=32, augment=False
)

Ultralytics 8.3.218 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=new_data, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=4, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=224, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11s-cls.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained=T

# Downloading & Reuploading best model

In [22]:
files.download('/content/runs/classify/train/weights/best.pt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [23]:
model = YOLO('/content/runs/classify/train/weights/best.pt')